<a href="https://colab.research.google.com/github/Ruby-L-i-n/CSDS234_FinalProject/blob/main/FeatureGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textblob

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

In [6]:
playlistDF = pd.read_csv("/content/drive/Shareddrives/CSDS 234 Final Project/track_data/tracks_info_0-9999.csv")

playlistDF

,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",69,0.904,0.813,4,-7.105,0,...,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4
1,Toxic,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,"['dance pop', 'pop']",79,0.774,0.838,5,-3.914,0,...,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4
2,Crazy In Love,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,"['pop', 'r&b']",83,0.664,0.758,2,-6.583,0,...,0.0598,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4
3,Rock Your Body,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,"['dance pop', 'pop']",79,0.892,0.714,4,-6.055,0,...,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4
4,It Wasn't Me,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,"['pop rap', 'reggae fusion']",70,0.853,0.606,0,-4.596,1,...,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34437,Diamond Child,Jon D,spotify:artist:1r2kTJ27zuaEoXasQT5NDd,['channel pop'],39,0.669,0.228,2,-12.119,1,...,0.0944,0.402,83.024,audio_features,3uCHI1gfOUL5j5swEh0TcH,spotify:track:3uCHI1gfOUL5j5swEh0TcH,https://api.spotify.com/v1/tracks/3uCHI1gfOUL5...,https://api.spotify.com/v1/audio-analysis/3uCH...,189184,4
34438,I Don't Know,Big Words,spotify:artist:5HCypjplgh5uQezvBpOfXN,['australian r&b'],34,0.493,0.727,1,-5.031,1,...,0.1290,0.289,73.259,audio_features,0P1oO2gREMYUCoOkzYAyFu,spotify:track:0P1oO2gREMYUCoOkzYAyFu,https://api.spotify.com/v1/tracks/0P1oO2gREMYU...,https://api.spotify.com/v1/audio-analysis/0P1o...,263680,4
34439,The Answer,Allan Rayman,spotify:artist:0sHN89qak07mnug3LVVjzP,['canadian contemporary r&b'],48,0.702,0.524,7,-10.710,1,...,0.2980,0.265,140.089,audio_features,2oM4BuruDnEvk59IvIXCwn,spotify:track:2oM4BuruDnEvk59IvIXCwn,https://api.spotify.com/v1/tracks/2oM4BuruDnEv...,https://api.spotify.com/v1/audio-analysis/2oM4...,189213,4
34440,25.22,Jon Jason,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,[],2,0.509,0.286,8,-14.722,1,...,0.1310,0.259,121.633,audio_features,4Ri5TTUgjM96tbQZd5Ua7V,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,https://api.spotify.com/v1/tracks/4Ri5TTUgjM96...,https://api.spotify.com/v1/audio-analysis/4Ri5...,194720,4
